In [1]:
import stageemi.dev.decorator_map as dm
import xarray as xr 
import ipywidgets as widg 
import ipyleaflet as ipyl
import datetime as dt
import os 
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl

In [2]:
ds_a = xr.open_dataset("../WWMF/20200126000000__PG0PAROME__WWMF__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc")

In [3]:
ds_a['latitude'] = ds_a.latitude.round(5)

In [4]:
ds_mask = xr.open_dataset("../GeoData/nc_departement/FRK24.nc")
ds_mask["latitude"] = ds_mask.latitude.round(5)

In [5]:
ds_d = ds_mask["mask"]*ds_a
ds = ds_d.squeeze("id")

In [8]:
@dm.gogeojson_wwmf
def get_step(step):
    return ds["unknown"].isel(step=step)

def update_html(feature, **kwargs):
    html1.value = '''
        <h4> Type de temps </h4>
        <h4><b>{}</b></h4>
    '''.format(feature['properties']['value'])
    
def on_change(change):
    geo_file,legend_file = get_step(change["new"])
    geojson_layer = ipyl.GeoJSON(data=geo_file,hover_style={"opacity":1})
    if len(m.layers) == 1:
        m.add_layer(geojson_layer)
    else: 
        m.substitute_layer(m.layers[1],geojson_layer)
    geojson_layer.on_hover(update_html)
    legend_file.seek(0)
    legend.value =legend_file.read() 
slider = widg.IntSlider(min=0,max=len(ds.step),step=1,value=0,description="step")
slider.observe(on_change,'value')

In [9]:
m = ipyl.Map(center=(45,0),zoom=7)
html1 = HTML('''
    <h4>Type de temps</h4>
    Hover over a pixel
''')
html1.layout.margin = '0px 20px 20px 20px'
control1 = WidgetControl(widget=html1, position='bottomright')
m.add_control(control1)
m.add_control(ipyl.WidgetControl(widget=slider,position="topright"))
geo_file,legend_file = get_step(0)
geojson_layer = ipyl.GeoJSON(data=geo_file,hover_style={"opacity":1,"fillColor":"#000000"})
geojson_layer.on_hover(update_html)
m.add_layer(geojson_layer)

In [10]:
legend_file.seek(0)
legend = widg.Image(layout=widg.Layout(height="430px"))
legend.value = legend_file.read() 
widg_map = widg.HBox([m,legend])
widg_map